# Dashboard both teams

This file contains the code of the dashboard of the masterthesis of Theo Vandeportaele, where players of both teams are displayed.

In [ ]:
import os
import glob
import threading
import dash
import json
from dash import html, dcc
from dash.dependencies import Input, Output, State, ClientsideFunction, MATCH, ALL
from PIL import Image
import io
import base64
from IPython.display import HTML
import time
from hashlib import sha512
import requests
import xml.etree.ElementTree as ET
from dash.exceptions import PreventUpdate
from functools import lru_cache
from apscheduler.schedulers.background import BackgroundScheduler

fixture_uuid = '9eldyiefgebuageoqyepzu0pg' 

In [ ]:
outlet_key = ''
outlet_secret = ''

# Get acces token to StatsPerform API
def retrieve_access_token():
    url = f'https://oauth.performgroup.com/oauth/token/{outlet_key}'
    params = {
        '_fmt': 'json',
        '_rt': 'b',
    }
    data = {
        'grant_type': 'client_credentials',
        'scope': 'b2b-feeds-auth',
    }

    timestamp = int(time.time() * 1000)
    base_token = f'{outlet_key}{timestamp}{outlet_secret}'
    token = sha512(base_token.encode()).hexdigest()

    headers = {
        'Timestamp': f'{timestamp}',
        'Authorization': f'Basic {token}',
        'Content-Type': 'application/x-www-form-urlencoded',
    }

    response = requests.post(url, headers=headers, data=data, params=params)
    response_json = response.json()

    return response_json['access_token']

contestant_home = ''
contestant_away = ''

# Get maps to map players from both teams from shirt_number to name
def retrieve_MA2(fixture_uuid): 
    global contestant_home
    global contestant_away 

    url = f'https://api.performfeeds.com/soccerdata/matchstats/{outlet_key}'
    params = {
        '_rt': 'b',
        '_fmt': 'xml',
        'fx': fixture_uuid
    }
    
    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    response = requests.get(url, headers=headers, params=params)
    MA2_data = response.text

    root = ET.fromstring(MA2_data)

    contestant_home = root.find('.//matchInfo//contestants')[0].get('id')
    contestant_away = root.find('.//matchInfo//contestants')[1].get('id')

    home_dict = {}
    away_dict = {}

    liveData = root.find('.//liveData')

    lineup_home = liveData.find(f'.//lineUp[@contestantId="{contestant_home}"]')
    lineup_away = liveData.find(f'.//lineUp[@contestantId="{contestant_away}"]')
    
    for player in lineup_home: 
        short_last_name = player.get('shortLastName')
        shirt_number = player.get('shirtNumber')

        if short_last_name and shirt_number: 
            home_dict[shirt_number] = short_last_name

    for player in lineup_away: 
        short_last_name = player.get('shortLastName')
        shirt_number = player.get('shirtNumber')

        if short_last_name and shirt_number: 
            away_dict[shirt_number] = short_last_name

    return home_dict, away_dict


access_token = retrieve_access_token()
home_dict, away_dict = retrieve_MA2(fixture_uuid)

# Get maps to go from name to shirt_number for both teams
def reverse_dict(input_dict):
    # Ensure values are unique, otherwise, this will raise a ValueError.
    if len(input_dict) != len(set(input_dict.values())):
        raise ValueError("Values in the dictionary must be unique to reverse the dictionary.")
    
    # Reverse the dictionary
    return {value: key for key, value in input_dict.items()}

reversed_home_dict = reverse_dict(home_dict)
reversed_away_dict = reverse_dict(away_dict)

# Get 2 list with all the players that play for both teams
shirt_numbers_home = []
shirt_numbers_away = []

for i in home_dict: 
    shirt_numbers_home.append(i)

for j in away_dict:
    shirt_numbers_away.append(j)

In [ ]:
# Create one list with all players that are playing for both teams
selected_players = []

for i in shirt_numbers_home: 
    selected_players.append(home_dict[i])

for j in shirt_numbers_away: 
    selected_players.append(away_dict[j])

# Retrieve info of the MA1 endpoint of the StatsPerform API
def retrieve_MA1(fixture_uuid): 

    try: 
        url = f'https://api.performfeeds.com/soccerdata/match/{outlet_key}'
        params = {
            '_rt': 'b',
            '_fmt': 'json',
            'fx': fixture_uuid
        }
        
        headers = {
            'Authorization': f'Bearer {access_token}'
        }
        
        response = requests.get(url, headers=headers, params=params)
    
        MA1_data = response.json()
        
        team1 = MA1_data['match'][0]['matchInfo']['contestant'][0]['name']
        team2 = MA1_data['match'][0]['matchInfo']['contestant'][1]['name']
        
        return f"{team1} vs {team2}"
    except: 
        return "" 

fixture_string = retrieve_MA1(fixture_uuid)    
print(fixture_string)

# Team int = 1 when Club Brugge is away, and 0 if they are at home
def get_team_int(): 
    team1 = retrieve_MA1(fixture_uuid)
    if team1.startswith('Club Brugge'):
        team = 'home'
        team_int = 0
    else: 
        team = 'away'
        team_int = 1
    return team_int

# Retrieve info of the MA2 endpoint of the StatsPerform API
def retrieve_MA2(fixture_uuid): 
    try: 
        url = f'https://api.performfeeds.com/soccerdata/matchstats/{outlet_key}'
        params = {
            '_rt': 'b',
            '_fmt': 'json',
            'fx': fixture_uuid
        }
        
        headers = {
            'Authorization': f'Bearer {access_token}'
        }
    
        response = requests.get(url, headers=headers, params=params)
        MA2_data = response.json()

        try: 
            score_string = f"{MA2_data['liveData']['goal'][-1]['homeScore']}-{MA2_data['liveData']['goal'][-1]['awayScore']}"
        except: 
            score_string = '0-0'

        ontarget_scoring_att_string = ''
        total_scoring_att_string = ''
        
        for team_int in range(2):  # Iterate over both teams
            try: 
                scoring_data = MA2_data['liveData']['lineUp'][team_int]['player']
                ontarget_scoring_att_total = 0
                total_scoring_att_total = 0

                # Iterate over player data
                for player in scoring_data:
                    # Initialize counts for the current player
                    ontarget_scoring_att_count = 0
                    total_scoring_att_count = 0

                    # Iterate over stats of the current player
                    for stat in player['stat']:
                        if stat['type'] == 'ontargetScoringAtt':
                            ontarget_scoring_att_count += int(stat['value'])
                        elif stat['type'] == 'totalScoringAtt':
                            total_scoring_att_count += int(stat['value'])

                    # Add counts to the total variables
                    ontarget_scoring_att_total += ontarget_scoring_att_count
                    total_scoring_att_total += total_scoring_att_count

                # Concatenate values with '-' between them
                if team_int == 0:
                    ontarget_scoring_att_string += str(ontarget_scoring_att_total)
                    total_scoring_att_string += str(total_scoring_att_total)
                else:
                    ontarget_scoring_att_string += f'-{ontarget_scoring_att_total}'
                    total_scoring_att_string += f'-{total_scoring_att_total}'

            except KeyError:  # If the key is not found
                pass  # Skip and move to the next team

        return [score_string, ontarget_scoring_att_string, total_scoring_att_string]
    except Exception as e: 
        print(e)
        return ['0-0', '0-0', '0-0']

# Retrieve info of the possession endpoint of the StatsPerform API
def retrieve_possession(fixture_uuid): 
    try: 
        url = f'https://api.performfeeds.com/soccerdata/possession/{outlet_key}'
        params = {
            '_rt': 'b',
            '_fmt': 'json',
            'fx': fixture_uuid
        }
        
        headers = {
            'Authorization': f'Bearer {access_token}'
        }
        
        response = requests.get(url, headers=headers, params=params)
        possession_data = response.json()
        pos_min = possession_data['liveData']['possession']['possessionWave'][0]['intervalLength'][1]['interval'][-1] # die 1 was eerst een 0
        return f"{pos_min['home']} - {pos_min['away']}"
    except:
        return "No possession data found"

# Retrieve info of the win probability endpoint of the StatsPerform API
def retrieve_win_prob(fixture_uuid): 
    try: 
        url = f'https://api.performfeeds.com/soccerdata/matchlivewinprobability/{outlet_key}'
        params = {
            '_rt': 'b',
            '_fmt': 'json',
            'fx': fixture_uuid
        }
        
        headers = {
            'Authorization': f'Bearer {access_token}'
        }
        
        response = requests.get(url, headers=headers, params=params)
        win_prob_data = response.json()
    
        win_prob_min = win_prob_data['liveData']['livePredictions'][-1]['prediction']
        
        # Initialize a dictionary to store probabilities
        probabilities = {}
        
        # Iterate over the data and store probabilities in the dictionary
        for item in win_prob_min:
            probabilities[item['type']] = item['probability']
        
        # Construct the result string based on the keys of the dictionary
        return f"{probabilities.get('Home', '0')} - {probabilities.get('Draw', '0')} - {probabilities.get('Away', '0')}"
    except: 
        return "No win probability data found"

# Retrieve info of the expected goals endpoint of the StatsPerform API
def retrieve_expected_goals(fixture_uuid): 

    url = f'https://api.performfeeds.com/soccerdata/matchexpectedgoals/{outlet_key}'
    params = {
        '_rt': 'b',
        '_fmt': 'xml',
        'fx': fixture_uuid
    }
    
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    
    response = requests.get(url, headers=headers, params=params)
    ratings_data = response.text
    
    team_int = get_team_int()

    # Parse the XML data
    root = ET.fromstring(ratings_data)
    
    # Find the matchInfo element
    match_info = root.find('.//liveData//events')

    xg_home = 0
    xg_away = 0

    for event in match_info.findall('.//event'):
        if team_int == 0:
            if event.get('contestantId') == '1oyb7oym5nwzny8vxf03szd2h': 
                qualifier = root.find('.//qualifier[@qualifierId="321"]')
                value = float(qualifier.get('value'))
                time = int(event.get('timeMin'))
                xg_home += value
            else: 
                xg_away += value
        else: 
            if event.get('contestantId') == '1oyb7oym5nwzny8vxf03szd2h': 
                qualifier = root.find('.//qualifier[@qualifierId="321"]')
                value = float(qualifier.get('value'))
                time = int(event.get('timeMin'))
                xg_away += value
            else: 
                xg_home += value

        
    return f'{round(xg_home, 2)} - {round(xg_away, 2)}'

# Retrieve info of the ratings endpoint of the StatsPerform API
def retrieve_ratings(fixture_uuid): 

    try: 
        url = f'https://api.performfeeds.com/soccerdata/matchplayerratings/{outlet_key}'
        params = {
            '_rt': 'b',
            '_fmt': 'xml',
            'fx': fixture_uuid
        }
        
        headers = {
            'Authorization': f'Bearer {access_token}'
        }
        
        response = requests.get(url, headers=headers, params=params)
        ratings_data = response.text
        
        team_int = get_team_int()
    
        # Parse the XML data
        root = ET.fromstring(ratings_data)
        
        # Find the matchInfo element
        match_info = root.find('.//playerRatings')[team_int]
        
        # Dictionary to store player IDs and scores
        player_scores = {}
        
        # Iterate through each player element
        for player_elem in match_info.findall('.//player'):
            # Get player ID
            player_id = player_elem.get('id')
            
            # Get score
            score_elem = player_elem.find('.//indexScore[@type="score"]')
            score = score_elem.text if score_elem is not None else None
            
            # Store player ID and score in the dictionary
            player_scores[player_id] = score
        
        # Create a new dictionary to store names and ratings
        player_ratings = {}
        
        # Replace the IDs with names
        for player_id, rating in player_scores.items():
            name = players_mapping.get(player_id)
            if name:
                player_ratings[name] = rating
    
        return player_ratings 
    except:
        return {}

# This function calculates the physical info of the player 
def physical_info_calculator(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)

    info_dict = {}
    for player_number, player_data in data.items():
        smoothed_array = player_data['smoothed_array']
        smoothed_count_array = player_data['smoothed_count_array']
        average_distance = player_data['average_distance']
        average_vel = player_data['average_vel']

        last_smoothed = smoothed_array[-1]
        last_smoothed_count = smoothed_count_array[-1]

        if contestant_home == '1oyb7oym5nwzny8vxf03szd2h':
            shirt_number_mapping_rev = home_dict
        else: 
            shirt_number_mapping_rev = away_dict

        info_dict[f"{shirt_number_mapping_rev[player_number]}"] = (last_smoothed - average_distance) + (last_smoothed_count - average_vel)
        
    # Sort the dictionary by values and return it
    sorted_dict = dict(sorted(info_dict.items(), key=lambda item: item[1]))
    return sorted_dict

# Get correct color for Fatigue window for every player
def get_rating_color(player_rating):
    rating = float(player_rating)
    if rating < 50:
        return 'red'
    elif 50 <= rating < 60:
        return 'orange'
    elif 60 <= rating < 70:
        return 'yellow'
    elif 70 <= rating < 80:
        return 'lightgreen'
    else:
        return 'darkgreen'
        
# Update the shirt_numbers when a player gets subbed in and out
def update_shirt_numbers(): 
    global shirt_numbers
    global selected_players

    
    url = f'https://api.performfeeds.com/soccerdata/match/{outlet_key}'
    params = {
        '_rt': 'b',
        '_fmt': 'xml',
        'fx': fixture_uuid
    }
    
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    
    response = requests.get(url, headers=headers, params=params)

    team_int = get_team_int()
    
    MA1_data = response.text

    root = ET.fromstring(MA1_data)
    
    match_info = root.find('.//liveData')

        
    for player_elem in match_info.findall('.//substitute'):
        if player_elem.get('contestantId') == contestant_home:
            player_on_id = player_elem.get('playerOnId')
            player_off_id = player_elem.get('playerOffId')

            player_on_name = players_mapping[player_on_id]
            player_off_name = players_mapping[player_off_id]

            player_on_number = shirt_number_mapping[player_on_name]
            player_off_number = shirt_number_mapping[player_off_name]

            if player_off_number in shirt_numbers_home: 
                index = shirt_numbers_home.index(player_off_number)
                shirt_numbers_home[index] = player_on_number
        elif player_elem.get('contestantId') == contestant_away:
            player_on_id = player_elem.get('playerOnId')
            player_off_id = player_elem.get('playerOffId')

            player_on_name = players_mapping[player_on_id]
            player_off_name = players_mapping[player_off_id]

            player_on_number = shirt_number_mapping[player_on_name]
            player_off_number = shirt_number_mapping[player_off_name]

            if player_off_number in shirt_numbers_away: 
                index = shirt_numbers_away.index(player_off_number)
                shirt_numbers_away[index] = player_on_number

        selected_players = []

        for i in shirt_numbers_home: 
            selected_players.append(home_dict[i])
        
        for j in shirt_numbers_away: 
            selected_players.append(away_dict[j])

# Initialise the values of the functions
previous_score_string , previous_ontarget_scoring_att_total , previous_total_scoring_att_total   = retrieve_MA2(fixture_uuid) 
previous_possession_string  = retrieve_possession(fixture_uuid)    
previous_win_prob_string  = retrieve_expected_goals(fixture_uuid)    
previous_ratings_data  = retrieve_ratings(fixture_uuid)    

In [ ]:
# Import necessary libraries
import os
import glob
from PIL import Image
from flask import Flask
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from functools import lru_cache
import threading
from apscheduler.schedulers.background import BackgroundScheduler
from IPython.display import display, HTML

# Initialize Dash app
app = dash.Dash(__name__)

# Function to get the newest images for a player
def get_newest_images(player_id):
    try:
        velocity_images = []
        distance_images = []
        player_folder = f"now_live/{player_id}"
        
        if os.path.exists(player_folder):
            # List image files in the player folder
            image_files = [f for f in os.listdir(player_folder) if f.endswith('.png')]
            
            # Separate velocity and distance images
            for image_file in image_files:
                if "_velocity_" in image_file:
                    velocity_images.append(image_file)
                elif "_distance_" in image_file:
                    distance_images.append(image_file)
            
            # Sort velocity and distance images based on the number at the end of the filename
            velocity_images.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]), reverse=True)
            distance_images.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]), reverse=True)
    
            # Get the top newest image for velocity and distance
            top_velocity_image = velocity_images[0] if velocity_images else None
            top_distance_image = distance_images[0] if distance_images else None
    
            # Open images using PIL
            if top_velocity_image:
                top_velocity_image = Image.open(os.path.join(player_folder, top_velocity_image))
            if top_distance_image:
                top_distance_image = Image.open(os.path.join(player_folder, top_distance_image))
    
        return top_distance_image, top_velocity_image
    except:
        print("No images found")

# Define the layout of the app
app.layout = html.Div([
    # Title div
    html.Div([
        html.H1(children=fixture_string, style={'textAlign': 'center', 'fontSize': '24px'})  # Set text size
    ]),
    
    html.Div([
        # Left part - dropdowns and images
        html.Div([
            dcc.Dropdown(
                id='dropdown-1',
                options=[{'label': player, 'value': player} for player in selected_players],
                value=selected_players[0] if selected_players else None,
                style={'fontSize': '16px'}  # Set text size
            ),
        
            html.Div(id='image-container-1'),
        
            dcc.Dropdown(
                id='dropdown-2',
                options=[{'label': player, 'value': player} for player in selected_players],
                value=selected_players[1] if len(selected_players) > 1 else None,
                style={'fontSize': '16px'}  # Set text size
            ),

            html.Div(id='image-container-2'),
            dcc.Interval(id='interval-component_img', interval=60*1000, n_intervals=0)  # Update every 60 seconds
            
        ], style={'width': '55%', 'display': 'inline-block', 'marginRight': '1%'}),  # Increased width for the left part

        # Right part - statistics
        html.Div([
            html.Div([
                dcc.Dropdown(
                    id='data-type-dropdown',
                    options=[
                        {'label': 'Fatigue', 'value': 'fatigue'},
                        {'label': 'Match Stats', 'value': 'match_stats'}
                    ],
                    value='fatigue',
                    style={'fontSize': '16px'}  # Set text size
                ),
                html.Div(id="statistics-output"),
                dcc.Interval(id='interval-component-stats', interval=60*1000, n_intervals=0),  # Interval component for stats update
                dcc.Interval(id='interval-component-ratings', interval=60*1000, n_intervals=0),  # Interval component for ratings update
                dcc.Interval(id='interval-component-file', interval=60*1000, n_intervals=0)  # Interval component for file update
            ]),
        ], style={'width': '40%', 'display': 'inline-block', 'vertical-align': 'top', 'marginLeft': '1%'}),  # Reduced width for the right part
    ])
], style={'fontSize': '16px'})  # Set text size for the entire layout

# Callback to update dropdown options for dropdown-1
@app.callback(
    Output('dropdown-1', 'options'),
    [Input('interval-component_img', 'n_intervals')]
)
def update_dropdown_1_options(n_intervals):
    # Your logic to update dropdown options goes here
    updated_options = [{'label': player, 'value': player} for player in selected_players]
    return updated_options

# Callback to update dropdown options for dropdown-2
@app.callback(
    Output('dropdown-2', 'options'),
    [Input('interval-component_img', 'n_intervals')]
)
def update_dropdown_2_options(n_intervals):
    # Your logic to update dropdown options goes here
    updated_options = [{'label': player, 'value': player} for player in selected_players]
    return updated_options

# Callbacks for updating images based on dropdown selection and intervals
@app.callback(
    Output('image-container-1', 'children'),
    [Input('dropdown-1', 'value'),
     Input('interval-component_img', 'n_intervals')]
)
def update_images_1(selected_player, n_intervals):
    if selected_player:
        images = get_newest_images(selected_player)
        if images:
            return [html.Img(src=image, style={'width': '45%', 'margin': '5px'}) for image in images]
        else:
            return html.Div("No images found", style={'color': 'red'})
    else:
        return []

@app.callback(
    Output('image-container-2', 'children'),
    [Input('dropdown-2', 'value'),
     Input('interval-component_img', 'n_intervals')]
)
def update_images_2(selected_player, n_intervals):
    if selected_player:
        images = get_newest_images(selected_player)
        if images:
            return [html.Img(src=image, style={'width': '45%', 'margin': '5px'}) for image in images]
        else:
            return html.Div("No images found", style={'color': 'red'})
    else:
        return []

# Caching function to retrieve data
@lru_cache(maxsize=None)  # Set maxsize=None for an unbounded cache
def retrieve_data_cached(fixture_uuid):
    score_string, ontarget_scoring_att_total, total_scoring_att_total = retrieve_MA2(fixture_uuid)
    possession_string = retrieve_possession(fixture_uuid)
    win_prob_string = retrieve_expected_goals(fixture_uuid)
    ratings_data = retrieve_ratings(fixture_uuid)
    return score_string, ontarget_scoring_att_total, total_scoring_att_total, possession_string, win_prob_string, ratings_data

# Function to update the cache
def update_cache():
    retrieve_data_cached.cache_clear()

# Callback for updating statistics based on dropdown selection and intervals
@app.callback(
    Output("statistics-output", "children"),
    [Input("data-type-dropdown", "value"),
     Input("interval-component-stats", "n_intervals"),
     Input("interval-component-ratings", "n_intervals"),
     Input("interval-component-file", "n_intervals")]  # Add this input for the file update interval
)
def update_statistics(data_type, n_intervals_stats, n_intervals_ratings, n_intervals_file):
    if data_type == "match_stats":
        score_string, ontarget_scoring_att_total, total_scoring_att_total, possession_string, win_prob_string, ratings_data = retrieve_data_cached(fixture_uuid)
        
        if not ratings_data:  # If ratings_data is empty
            return html.Div([
                html.Div([
                    html.Div(style={'height': '15px'}),  
                    html.Div([
                        html.Div("Score:", style={'font-weight': 'bold', 'font-size': '17px'}),
                        html.Div("Scoring Attempts:", style={'font-weight': 'bold', 'font-size': '17px'}),
                        html.Div("On-Target Scoring Attempts:", style={'font-weight': 'bold', 'font-size': '17px'}),
                        html.Div("Possession:", style={'font-weight': 'bold', 'font-size': '17px'}),
                        html.Div("Expected goals:", style={'font-weight': 'bold', 'font-size': '17px'})
                    ], style={'width': '65%', 'text-align': 'left', 'float': 'left'}),
                    html.Div([
                        html.Div(score_string, style={'font-size': '17px'}),
                        html.Div(total_scoring_att_total, style={'font-size': '17px'}),
                        html.Div(ontarget_scoring_att_total, style={'font-size': '17px'}),
                        html.Div(possession_string, style={'font-size': '17px'}),
                        html.Div(win_prob_string, style={'font-size': '17px'})
                    ], style={'width': '35%', 'text-align': 'left', 'float': 'left'})
                ], style={'overflow': 'hidden', 'margin-bottom': '20px'})])

        # Sort the ratings data by rating in descending order
        try:
            sorted_ratings_data = sorted(ratings_data.items(), key=lambda x: float(x[1]), reverse=True)
            filtered_ratings_data = [(player_name, rating) for player_name, rating in sorted_ratings_data if shirt_number_mapping[player_name] in shirt_numbers]
            
            return html.Div([
                html.Div([
                    html.Div(style={'height': '15px'}),  
                    html.Div([
                        html.Div("Score:", style={'font-weight': 'bold', 'font-size': '17px'}),
                        html.Div("Scoring Attempts:", style={'font-weight': 'bold', 'font-size': '17px'}),
                        html.Div("On-Target Scoring Attempts:", style={'font-weight': 'bold', 'font-size': '17px'}),
                        html.Div("Possession:", style={'font-weight': 'bold', 'font-size': '17px'}),
                        html.Div("Expected goals:", style={'font-weight': 'bold', 'font-size': '17px'})
                    ], style={'width': '65%', 'text-align': 'left', 'float': 'left'}),
                    html.Div([
                        html.Div(score_string, style={'font-size': '17px'}),
                        html.Div(total_scoring_att_total, style={'font-size': '17px'}),
                        html.Div(ontarget_scoring_att_total, style={'font-size': '17px'}),
                        html.Div(possession_string, style={'font-size': '17px'}),
                        html.Div(win_prob_string, style={'font-size': '17px'})
                    ], style={'width': '35%', 'text-align': 'left', 'float': 'left'})
                ], style={'overflow': 'hidden', 'margin-bottom': '20px'}),
                html.Hr(),
                html.Div([
                    html.Div("Ratings:", style={'font-weight': 'bold', 'font-size': '22px'}),
                    html.Div([
                        html.Div("Player", style={'font-weight': 'bold', 'font-size': '18px'}),
                        html.Div("Rating", style={'font-weight': 'bold', 'font-size': '18px'})
                    ], style={'display': 'grid', 'grid-template-columns': '1fr 1.5fr'}),
                    *[html.Div([
                        html.Div(player_name, style={'font-size': '18px', 'background-color': get_rating_color(player_rating), 'border': '1px solid black'}),
                        html.Div(player_rating, style={'font-size': '18px', 'background-color': get_rating_color(player_rating), 'border': '1px solid black'})
                    ], style={'display': 'grid', 'grid-template-columns': '1fr 1.5fr'}) for player_name, player_rating in filtered_ratings_data]
                ])  # Display ratings under the title
            ])
        except:
            return html.Div([
                html.Div([
                    html.Div(style={'height': '15px'}),  
                    html.Div([
                        html.Div("Score:", style={'font-weight': 'bold', 'font-size': '17px'}),
                        html.Div("Scoring Attempts:", style={'font-weight': 'bold', 'font-size': '17px'}),
                        html.Div("On-Target Scoring Attempts:", style={'font-weight': 'bold', 'font-size': '17px'}),
                        html.Div("Possession:", style={'font-weight': 'bold', 'font-size': '17px'}),
                        html.Div("Expected goals:", style={'font-weight': 'bold', 'font-size': '17px'})
                    ], style={'width': '65%', 'text-align': 'left', 'float': 'left'}),
                    html.Div([
                        html.Div(score_string, style={'font-size': '17px'}),
                        html.Div(total_scoring_att_total, style={'font-size': '17px'}),
                        html.Div(ontarget_scoring_att_total, style={'font-size': '17px'}),
                        html.Div(possession_string, style={'font-size': '17px'}),
                        html.Div(win_prob_string, style={'font-size': '17px'})
                    ], style={'width': '35%', 'text-align': 'left', 'float': 'left'})
                ], style={'overflow': 'hidden', 'margin-bottom': '20px'})])

    elif data_type == "fatigue":
        # Find the physical data files and sort them based on the number at the end
        files = glob.glob("now_live/physical_data/physical_data_*.json")

        if not files:  # If no physical data files are found
            return "No physical data available"  # Return the message "No physical data available"
        
        newest_file = max(files, key=lambda x: int(x.split('_')[-1].split('.')[0]))
        
        # Calculate physical info using the newest file
        physical_info = physical_info_calculator(newest_file)

        # Create a list of HTML elements for each player in the physical info dictionary

        if contestant_home == '1oyb7oym5nwzny8vxf03szd2h':
            shirt_number_mapping = reversed_home_dict 
        else: 
            shirt_number_mapping = reversed_away_dict 
        
        player_elements = [
            html.Div([
                html.Div(player_name, style={'font-size': '20px', 'color': 'black', 'text-align': 'left'}),
                # Add more elements here if needed
            ], style={
                'border': '1px solid black',
                'padding': '10px',
                'margin-bottom': '10px',
                'background-color': (
                    'green' if physical_info[player_name] >= 100 else
                    'yellow' if physical_info[player_name] >= 50 else
                    'orange' if physical_info[player_name] >= 0 else
                    'red'
                )
            }) 
            for player_name in physical_info.keys() if player_name in away_dict.values()
        ]

        return html.Div([
            html.Div("Physical Info for Players:", style={'font-weight': 'bold', 'font-size': '22px'}),
            *player_elements
        ])
    else:
        return "Fatigue statistics will be displayed here when selected."

# Scheduler to update the cache every minute
scheduler = BackgroundScheduler()
scheduler.add_job(update_cache, 'interval', minutes=1)
scheduler.start()

# Scheduler to update the shirt_numbers every minute (for subs)
shirt_number_scheduler = BackgroundScheduler()
shirt_number_scheduler.add_job(update_shirt_numbers, 'interval', minutes=1)
shirt_number_scheduler.start()

# Function to run the Dash app in a separate thread
def run_dash_app():
    app.run_server(mode='inline', port=8050)

# Start the Dash app in a separate thread
thread = threading.Thread(target=run_dash_app)
thread.start()

# Display the link to open the external window
display(HTML("<a href='http://127.0.0.1:8050/' target='_blank'>Open Dashboard in External Window</a>"))